In [3]:
import pandas as pd

# Fichiers à concaténer
p1 = "page_BNI.csv"
p2 = "page_sgci.csv"
p3 = "page_ecobank.csv"
p4 = "page_NSIA.csv"
p5 = "page_BACI.csv"

# Chargement des fichiers
df1 = pd.read_csv(p1)
df2 = pd.read_csv(p2)
df3 = pd.read_csv(p3)
df4 = pd.read_csv(p4)
df5 = pd.read_csv(p5)

# Concaténation
df_concat = pd.concat([df1, df2, df3, df4,df5], ignore_index=True)

# Optionnel : supprimer les doublons
#df_concat.drop_duplicates(inplace=True)

# Enregistrement dans un nouveau fichier
df_concat.to_csv("facebook_commentaires_concatene.csv", index=False)

print("✅ Fichiers concaténés avec succès dans 'facebook_commentaires_concatene.csv'")


✅ Fichiers concaténés avec succès dans 'facebook_commentaires_concatene.csv'


In [ ]:
from flask import Flask, request, jsonify
import random

app = Flask(__name__)

# --------------------------
# Base de 60 questions QCM
# --------------------------
questions = [
    # ---------------- Python / Intro ML ----------------
    {"id": 1, "question": "Quels types sont mutables en Python ?", "options": ["list", "tuple", "dict", "set", "str"], "answers": [0, 2, 3]},
    {"id": 2, "question": "Quels algorithmes sont supervisés ?", "options": ["KMeans", "Régression logistique", "SVM", "Random Forest", "PCA"], "answers": [1, 2, 3]},
    {"id": 3, "question": "Quels types de boucles existent en Python ?", "options": ["for", "while", "repeat", "loop", "do-while"], "answers": [0, 1]},

    # ---------------- Machine Learning ----------------
    {"id": 4, "question": "Quelles sont des méthodes de réduction de dimension ?", "options": ["ACP (PCA)", "t-SNE", "Random Forest", "UMAP", "KMeans"], "answers": [0,1,3]},
    {"id": 5, "question": "Quels sont des algorithmes de clustering ?", "options": ["KMeans", "DBSCAN", "Agglomératif", "Régression linéaire", "Isolation Forest"], "answers": [0,1,2]},
    {"id": 6, "question": "Quelles métriques conviennent pour classifier un dataset déséquilibré ?", "options": ["Accuracy", "F1-score", "AUC-ROC", "Précision", "Rappel"], "answers": [1,2,4]},
    {"id": 7, "question": "Quelles sont des méthodes de régularisation ?", "options": ["L1 (Lasso)", "L2 (Ridge)", "Dropout", "BatchNorm", "None"], "answers": [0,1]},
    {"id": 8, "question": "Quels modèles sont adaptés à la régression ?", "options": ["Régression linéaire", "Régression logistique", "SVR", "Réseaux de neurones", "Naive Bayes"], "answers": [0,2,3]},

    # ---------------- Statistiques ----------------
    {"id": 9, "question": "Quelles sont des mesures de tendance centrale ?", "options": ["Moyenne", "Médiane", "Mode", "Variance", "Écart-type"], "answers": [0,1,2]},
    {"id": 10, "question": "Quelles sont des mesures de dispersion ?", "options": ["Écart-type", "Variance", "Médiane", "Amplitude", "Mode"], "answers": [0,1,3]},
    {"id": 11, "question": "Quels tests sont paramétriques ?", "options": ["t-test", "ANOVA", "Chi2", "Mann-Whitney", "Kruskal-Wallis"], "answers": [0,1]},
    {"id": 12, "question": "Quels sont des tests non paramétriques ?", "options": ["Mann-Whitney", "Wilcoxon", "Kruskal-Wallis", "ANOVA", "t-test"], "answers": [0,1,2]},

    # ---------------- Deep Learning ----------------
    {"id": 13, "question": "Quels sont des types de réseaux de neurones ?", "options": ["CNN", "RNN", "GAN", "SVM", "MLP"], "answers": [0,1,2,4]},
    {"id": 14, "question": "Quelles sont des fonctions d’activation non linéaires ?", "options": ["ReLU", "Sigmoïde", "Tanh", "Softmax", "Dropout"], "answers": [0,1,2]},
    {"id": 15, "question": "Quelles techniques réduisent l’overfitting ?", "options": ["Dropout", "Régularisation L2", "Augmentation de données", "BatchNorm", "Augmenter la profondeur"], "answers": [0,1,2]},
    {"id": 16, "question": "Quelles sont des architectures populaires en vision ?", "options": ["ResNet", "VGG", "Inception", "BERT", "EfficientNet"], "answers": [0,1,2,4]},

    # ---------------- IA générative ----------------
    {"id": 17, "question": "Quels modèles sont utilisés en IA générative ?", "options": ["GAN", "VAE", "Transformers", "Random Forest", "CNN"], "answers": [0,1,2]},
    {"id": 18, "question": "Quels sont des usages de l’IA générative ?", "options": ["Génération d’images", "Traduction automatique", "Synthèse vocale", "Filtrage anti-spam", "Création de texte"], "answers": [0,1,2,4]},
    {"id": 19, "question": "Quels sont des modèles de langage ?", "options": ["BERT", "GPT", "T5", "AlexNet", "RoBERTa"], "answers": [0,1,2,4]},

    # ... (compléter jusqu'à 60)
]

# --------------------------
# Endpoint pour récupérer toutes les questions avec options randomisées
# --------------------------
@app.route("/questions", methods=["GET"])
def get_questions():
    safe_questions = []

    for q in questions:
        q_copy = q.copy()
        q_copy.pop("answers", None)  # Ne pas exposer les bonnes réponses

        # Randomiser l'ordre des options et mettre à jour les indices corrects
        options = q_copy["options"]
        indices = list(range(len(options)))
        combined = list(zip(options, indices))
        random.shuffle(combined)
        shuffled_options, old_indices = zip(*combined)
        q_copy["options"] = list(shuffled_options)

        # Calculer nouvelle position des réponses correctes
        correct_set = set(q["answers"])
        new_answers = [i for i, idx in enumerate(old_indices) if idx in correct_set]
        q_copy["shuffled_answers"] = new_answers  # frontend peut utiliser pour validation si besoin

        safe_questions.append(q_copy)

    return jsonify(safe_questions)

# --------------------------
# Endpoint pour soumettre les réponses
# --------------------------
@app.route("/submit_quiz", methods=["POST"])
def submit_quiz():
    data = request.json
    user_answers = data.get("answers")  # format: [{ "id": 1, "selected": [0,2,3] }, ...]

    if not user_answers:
        return jsonify({"error": "Pas de réponses fournies"}), 400

    total_score = 0
    total_questions = len(questions)
    corrections = []

    for ua in user_answers:
        q = next((q for q in questions if q["id"] == ua["id"]), None)
        if not q:
            continue
        user_set = set(ua.get("selected", []))
        correct_set = set(q["answers"])

        is_correct = user_set == correct_set
        if is_correct:
            total_score += 1

        corrections.append({
            "id": q["id"],
            "question": q["question"],
            "user_selected": ua.get("selected", []),
            "correct_answers": q["answers"],
            "is_correct": is_correct
        })

    return jsonify({
        "score": total_score,
        "total": total_questions,
        "success_rate": f"{(total_score/total_questions)*100:.2f}%",
        "corrections": corrections
    })


if __name__ == "__main__":
    app.run(debug=True)
  